# Croatian Sentiment Classification the old-fashioned way: 
## `Naive Bayes`, (`Logistic Regression`, and `Ngrams`)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *
from fastai.data.all import *
import torch
from pathlib import Path
from fastdownload import download_url 
from fastai.text.data import *
from fastai.text.core import *
from fastai.text.all import *

In [2]:
data_path = Path("./data/")
lang = 'hr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
mdl_path = path/'models'
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
lm_fns = [os.getcwd()/mdl_path/lm_fns[0], os.getcwd()/mdl_path/lm_fns[1]]

In [4]:
from nlputils import split_wiki,get_wiki
docs_path = split_wiki(path, 'hr')
docs_path

data/hrwiki/docs already exists; not splitting


Path('data/hrwiki/docs')

In [5]:
(path/"filtered").exists()

True

In [18]:
train_df = pd.read_csv(path/'train.csv')
test_df = pd.read_csv(path/'test.csv')
df = pd.concat([train_df,test_df], sort=False)
print(len(train_df), len(test_df))
test_df.head()

2836 708


,0,1
0,False,"Čekali smo konobara 15 min da bi napokon došao i rekao da nema gotovo ničega od dnevne ponude gableca, na što smo ga zamolili da donese jelovnik koji nije stigao niti nakon 10 min. Loša organizacija."
1,False,"Koktelima baš nisam bila zadovoljna, ali hrana jako dobra. Usluga bi mogla biti brža."
2,True,"Konoba Galija oduševljava svojim raznovrsnim jelovnikom u svako doba godine, a pritom ukusno pripremljenom hranom, savršenim desertima te ljubaznim osobljem..Moram izdvojiti predjelo Pjat s plodovima mora koje je, osim ukusno, predivno servirano za one koje cijene i estetiku. Naravno da je i u tom stilu i glavno jelo.Slavonka koja će se s obitelji uvijek vratiti ovoj konobi 🤗🤗"
3,False,Naručili smo tunu s blitvom i krumpirom.Tuna je bila dosta slana pa smo rekli osoblju.Bila je u umaku od soje.Jedina zamjerka bi bila kad smo tražili preporuku da nam je osoblje reklo da je u umaku od soje.To nam ne odgovora jer sa grila nam je tuna s maslin.uljem.Osoblje je bilo brzo i ljubazno.
4,True,Jedan od najboljih kebaba u gradu. Isplate se te dvije kune koliko je skuplji.


In [112]:
tok = SentencePieceTokenizer(special_toks=[], lang='hr')
dls = TextDataLoaders.from_df(train_df, valid_pct=0.2, 
                              is_lm=False, 
                              bs=64, 
                              text_col=1,
                              label_col=0,
                             )

In [119]:
TokenCounter = lambda review_index : Counter((dls.train_ds)[review_index][0].tolist())
TokenCounter(39).items()

dict_items([(2, 1), (8, 1), (14, 1), (12, 1), (32, 1), (9, 2), (18, 1), (133, 1), (45, 1), (29, 1), (10, 1)])

In [242]:
itos = dls.vocab[0]


In [142]:
n_terms = len(dls.vocab[0])
n_docs = len(dls.train_ds)
make_token_counter = lambda review_index : Counter((dls.train_ds)[review_index][0].tolist())
def count_vectorizer(review_index,n_terms = n_terms,make_token_counter = make_token_counter):
    # input: review index, n_terms, and tokenizer function
    # output: embedding vector for the review
    embedding_vector = np.zeros(n_terms)        
    keys = list(make_token_counter(review_index).keys())
    values = list(make_token_counter(review_index).values())
    embedding_vector[keys] = values
    return embedding_vector

# make the embedding vector for the first review
embedding_vector = count_vectorizer(0)
n_docs

2269

In [143]:
print(f'The review is embedded in a {len(embedding_vector)} dimensional vector')
embedding_vector

The review is embedded in a 2544 dimensional vector


array([16.,  0.,  1., ...,  0.,  0.,  0.])

In [144]:
# Define a function to build the full document-term matrix
print(f'there are {n_docs} reviews, and {n_terms} unique tokens in the vocabulary')
def make_full_doc_term_matrix(count_vectorizer,n_terms=n_terms,n_docs=n_docs):
    
    # loop through the movie reviews
    for doc_index in range(n_docs):
        
        # make the embedding vector for the current review
        embedding_vector = count_vectorizer(doc_index,n_terms)    
            
        # append the embedding vector to the document-term matrix
        if(doc_index == 0):
            A = embedding_vector
        else:
            A = np.vstack((A,embedding_vector))
            
    # return the document-term matrix
    return A

# Build the full document term matrix for the movie_reviews training set
A = make_full_doc_term_matrix(count_vectorizer)

there are 2269 reviews, and 2544 unique tokens in the vocabulary


In [145]:
NNZ = np.count_nonzero(A)
sparsity = (A.size-NNZ)/A.size
print(f'Only {NNZ} of the {A.size} elements in the document-term matrix are nonzero')
print(f'The sparsity of the document-term matrix is {sparsity}')
A

Only 46005 of the 5772336 elements in the document-term matrix are nonzero
The sparsity of the document-term matrix is 0.9920300897245067


array([[16.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  1., ...,  0.,  0.,  0.],
       [ 4.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 5.,  0.,  1., ...,  0.,  0.,  0.],
       [ 6.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [148]:
for i in dls.train_ds[0][0]:
    print(dls.vocab[0][i], end= " ")

xxbos xxmaj ambijent je ugodan , pogled na more sa xxunk xxunk je xxunk . xxmaj osoblje je susretljivo te uvijek stoje na xxunk , vlasnik je ugodan i xxunk . xxmaj hrana je zaista izvrsna , meso je uvijek pečeno onako kako se xxunk da bude . xxmaj xxunk jela i ostali plodovi mora su jako ukusni i uvijek xxunk topla preporuka svima koji žele dobro pojesti uz sasvim xxunk cijene . xxmaj takoder obavezno narucite juhu od xxunk kao toplo predjelo , bez obzira xxunk li ostati na xxunk ili xxunk xxunk za glavno xxunk xxunk kad smo u blizini . 

## Naivni Bayesov klasifikator

<!-- #### Then, given the knowledge that a review is classified as `positive`, the `conditional likelihood` that a token $t$ will appear in the review is -->
### $ L(t|+) = \frac{C^{+}_{t}}{N^+}$,  $ L(t|-) = \frac{C^{-}_{t}}{N^-}$, $ R_{t} = \text{log} \frac{L(t|+)}  {L(t|-)}$
<!-- #### and simlarly, the `conditional likelihood` of a token appearing in a `negative` review is  -->

<!-- #### From the class likelihood ratios, we can define a **log-count ratio** $R_{t}$ for each token $t$ as -->
<!-- ###  -->
<!-- #### The `log-count ratio` ranks tokens by their relative affinities for positive and negative reviews -->
<!-- #### We observe that
* $R_{t} \gt 0$ means `positive` reviews are more likely to contain this token 
* $R_{t} \lt 0$ means `negative` reviews are more likely to contain this token 
* $R_{t} = 0$ indicates the token $t$ has equal likelihood to appear in  `positive` and `negative` reviews -->

#### The log probability ratio is

### $b = \text{log} \frac{p} {q}$ 

<!-- #### is a measure of the `bias`, or `imbalance` in the data set. 

* $b = 0$ indicates a perfectly balanced data set
* $b \gt 0$ indicates bias towards `positive` reviews 
* $b \lt 0$ indicates bias towards `negative` reviews  -->

In [210]:
pos_labels = np.array(list(map(lambda x: x[1], dls.train_ds)))
neg_labels = 1-pos_labels
neg_labels

array([0, 0, 0, ..., 0, 0, 0])

In [238]:
L_pos = (pos_labels[None,:]@A)[0,:]
L_pos = (L_pos + 1) / (L_pos.sum() + 1)
L_neg = (neg_labels[None,:]@A)[0,:]
L_neg = (L_neg + 1) / (L_neg.sum() + 1)
R = np.log(L_pos / L_neg)

In [244]:
n_tokens = 10
highest_R = np.argpartition(R, -n_tokens)[-n_tokens:]
lowest_R = np.argpartition(R, n_tokens)[:n_tokens]
print(f'Highest {n_tokens} log-count ratios: {R[list(highest_R)]}\n')
print(f'Lowest {n_tokens} log-count ratios: {R[list(lowest_R)]}')

Highest 10 log-count ratios: [2.5204758  2.5204758  2.58110042 2.6195667  2.63825884 2.63825884
 2.63825884 2.74361935 3.02503181 2.95333988]

Lowest 10 log-count ratios: [-2.73701957 -2.89117025 -2.69445996 -3.51020946 -2.55469801 -3.08532627
 -2.8670727  -2.65000819 -2.73701957 -2.65000819]


In [247]:
print("Pozitivne rijeci:", [itos[k] for k in highest_R])
print("Negativne rijeci:", [itos[k] for k in lowest_R])

Pozitivne rijeci: ['vrhunsko', 'kvalitetna', 'prefina', 'svaka', 'vraćamo', 'vidimo', '🔝', 'najboljih', 'pohvale', 'vrhunska']
Negativne rijeci: ['umjesto', 'cijenu', 'loše', 'katastrofa', 'bezobrazno', 'loša', 'vise', 'zvjezdice', 'nitko', 'naručili']


In [249]:
b = L_pos.sum() / L_neg.sum()

In [274]:
def f(pair):
    sent = pair[0]
    return R[sent].sum() + b > 0, pair[1]
preds = np.array(list(map(f, dls.valid_ds)))
(preds[:, 0] == preds[:, 1]).sum() / len(preds)

0.8306878306878307